In [1]:
import requests
import pandas as pd

dataframes = [pd.DataFrame() for _ in range(12)]


#base_url
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

#query
query="/api/3/action/package_show"

url = base_url + query

params = {"id": "budget-operating-budget-program-summary-by-expenditure-category"}

package = requests.get(url, params=params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()
        dURL = str(resource_metadata["result"]["url"])

        # Check if the URL contains years from 2014 to 2023
        if any(str(year) in resource_metadata["result"]["url"] for year in range(2014, 2024)):
            
            if (("preliminary" not in dURL) and ("recommended" not in dURL)):
                
                # try different sheet names:
                # Option 1 = sheet by order 0
                # Option 2 = Open Data
                # Option 3 = Year
                try:
                    df2 = pd.read_excel(resource_metadata["result"]["url"], sheet_name=0, header=0)
                    dataframes[idx] = df2
                    
                    if df2.empty:
                        df3 = pd.read_excel(resource_metadata["result"]["url"], sheet_name='Open Data', header=0)
                        dataframes[idx] = df3
                        
                    print(f"DF created: {idx} - {dURL}")
                        
                except:
                    for yr in range(2014, 2024):
                        try:
                            df = pd.read_excel(resource_metadata["result"]["url"], sheet_name=str(yr), header=0)
                            dataframes[idx] = df
                            print(f"DF created: {idx} - {dURL}")
                            break
                        except:
                            print(f"Sheetname not found: {yr}")

DF created: 0 - https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/a6f7a8e8-e497-4f77-9881-daba429ea981/download/approved-operating-budget-summary-2023.xlsx
DF created: 1 - https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/9e5f9a63-fdeb-46e4-9f5f-8143038de56d/download/approved-operating-budget-summary-2022.xlsx
Sheetname not found: 2014
Sheetname not found: 2015
Sheetname not found: 2016
Sheetname not found: 2017
Sheetname not found: 2018
Sheetname not found: 2019
Sheetname not found: 2020
DF created: 2 - https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/930502fa-87f1-4d71-8b71-4435df594b38/download/2021-open-data2.xlsx
DF created: 3 - https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/4e58558a-4773-4cd0-a16a-de481f25cb4d/download/2020-operating-open-data.xlsx
Sheetname not found: 20

C:\Users\lukas\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


DF created: 10 - https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c90a5d3-5598-4c02-abf2-169456c8f1f1/resource/7e23b244-91a7-4e70-9baf-be231ff58d59/download/approved-operating-budget-summary-2014.xlsx


In [2]:
print("-"*30)
for df in dataframes:
    print(df.columns)
    print(f"DF Size = {df.shape[0]} Rows.")
    print("-"*20)

------------------------------
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2023],
      dtype='object')
DF Size = 20127 Rows.
--------------------
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2022],
      dtype='object')
DF Size = 19714 Rows.
--------------------
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2021],
      dtype='object')
DF Size = 20816 Rows.
--------------------
Index([          'Program',           'Service',          'Activity',
         'Expense/Revenue',     'Category Name', 'Sub-Category Name',
         'Commitment item',                2020],
      dtype='o

In [3]:
budget2014_df = dataframes[10]
budget2015_df = dataframes[9]
budget2016_df = dataframes[7] 
budget2017_df = dataframes[6] 
budget2018_df = dataframes[5] 
budget2019_df = dataframes[4] 
budget2020_df = dataframes[3] 
budget2021_df = dataframes[2] 
budget2022_df = dataframes[1]
budget2023_df = dataframes[0]

#create new List to maintain original dataframes
df_list = [
    budget2014_df,
    budget2015_df,
    budget2016_df,
    budget2017_df,
    budget2018_df,
    budget2019_df,
    budget2020_df,
    budget2021_df,
    budget2022_df,
    budget2023_df,
]

In [4]:
#Renaming the last column using a loop
for i, df in enumerate(df_list):
    year = 2014 + i
    new_column_name = f'Budgeted amount {year}'
    df.rename(columns={year: new_column_name}, inplace=True)

In [5]:
budget2016_df.rename(columns={" 2016": "Budgeted amount 2016"}, inplace=True)

In [6]:
#Changing the scientific notation 
format_sci_notation = lambda x: '{:.6e}'.format(float(x))
for df in df_list:
    last_col = df.columns[-1]
    df[last_col] = pd.to_numeric(df[last_col], errors='coerce')

In [7]:
#clean 2021 budget category "0"
budget2021_df = budget2021_df.loc[budget2021_df["Category Name"] != 0, :]

In [8]:
#Seperating the dataframes into expense and revenue dataframes
budget2014_df_exp = budget2014_df[budget2014_df['Expense/Revenue']=='Expenses']
budget2015_df_exp = budget2015_df[budget2015_df['Expense/Revenue']=='Expenses']
budget2016_df_exp = budget2016_df[budget2016_df['Expense/Revenue']=='Expenses']
budget2017_df_exp = budget2017_df[budget2017_df['Expense/Revenue']=='Expenses']
budget2018_df_exp = budget2018_df[budget2018_df['Expense/Revenue']=='Expenses']
budget2019_df_exp = budget2019_df[budget2019_df['Expense/Revenue']=='Expenses']
budget2020_df_exp = budget2020_df[budget2020_df['Expense/Revenue']=='Expenses']
budget2021_df_exp = budget2021_df[budget2021_df['Expense/Revenue']=='Expenses']
budget2022_df_exp = budget2022_df[budget2022_df['Expense/Revenue']=='Expenses']
budget2023_df_exp = budget2023_df[budget2023_df['Expense/Revenue']=='Expenses']

#Revenues
budget2014_df_rev = budget2014_df[budget2014_df['Expense/Revenue']=='Revenues']
budget2015_df_rev = budget2015_df[budget2015_df['Expense/Revenue']=='Revenues']
budget2016_df_rev = budget2016_df[budget2016_df['Expense/Revenue']=='Revenues']
budget2017_df_rev = budget2017_df[budget2017_df['Expense/Revenue']=='Revenues']
budget2018_df_rev = budget2018_df[budget2018_df['Expense/Revenue']=='Revenues']
budget2019_df_rev = budget2019_df[budget2019_df['Expense/Revenue']=='Revenues']
budget2020_df_rev = budget2020_df[budget2020_df['Expense/Revenue']=='Revenues']
budget2021_df_rev = budget2021_df[budget2021_df['Expense/Revenue']=='Revenues']
budget2022_df_rev = budget2022_df[budget2022_df['Expense/Revenue']=='Revenues']
budget2023_df_rev = budget2023_df[budget2023_df['Expense/Revenue']=='Revenues']

In [10]:
budget2023_df_rev.head(10)

,Program,Service,Activity,Expense/Revenue,Category Name,Sub-Category Name,Commitment item,Budgeted amount 2023
91,311 Toronto,311 Development,311 PMO,Revenues,Transfers From Capital,Transfers From Capital,Transfers From Capital Fund,-1745921.89
92,311 Toronto,311 Service Delivery,311 Service Delivery,Revenues,Contribution From Reserves/Reserve Funds,Contribution From Reserves/Reserve Funds,Contributions From Reserve Funds,-290495.00
93,311 Toronto,311 Service Delivery,311 Service Delivery,Revenues,Inter-Divisional Recoveries,Inter-Divisional Recoveries,IDR - Public Health,-169235.00
94,311 Toronto,311 Service Delivery,311 Service Delivery,Revenues,Inter-Divisional Recoveries,Inter-Divisional Recoveries,IDR - Solid Waste Mgmt Services,-4208952.00
95,311 Toronto,311 Service Delivery,311 Service Delivery,Revenues,Inter-Divisional Recoveries,Inter-Divisional Recoveries,IDR - Water Services,-2282813.00
96,311 Toronto,311 Service Delivery,311 Service Delivery,Revenues,Inter-Divisional Recoveries,Inter-Divisional Recoveries,IDR -Shelter Support & Housing Admin,-162966.00
97,311 Toronto,311 Service Delivery,311 Service Delivery,Revenues,Other Revenue,Miscellaneous Revenues,Sundry Revenue,-18000.00
126,Arena Boards of Management,Community Ice & Facility Booking,Not assigned,Revenues,User Fees & Donations,User Fees,Advertising Contract Revenues,-15000.00
127,Arena Boards of Management,Community Ice & Facility Booking,Not assigned,Revenues,Contribution From Reserves/Reserve Funds,Contribution From Reserves/Reserve Funds,Contributions From Reserve Funds,0.00
128,Arena Boards of Management,Community Ice & Facility Booking,Not assigned,Revenues,Contribution From Reserves/Reserve Funds,Contribution From Reserves/Reserve Funds,Contributions From Reserves,0.00
